In [2]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd

df_nombres = pd.read_csv('nombres_productos.csv')
df_inversiones = pd.read_excel('inversiones_helados.xlsx')
df_ventas = pd.read_excel('ventas_helados.xlsx')

### Acceso a los datos de ventas

Carga el dataset de ventas. ¿Cuántas unidades de cualquier helado se han vendido en total a lo largo del histórico?

In [4]:
df_ventas.sample(10)

,fecha,id_producto,ventas_unidades,precio
783,2019-12-09,7118,0,NaN
1265,2020-10-12,2033,1668,2.49
857,2020-01-20,402,2014,2.96
700,2019-11-04,4468,228,2.54
322,2019-05-27,8689,7920,2.44
227,2019-04-15,9134,6523,2.41
954,2020-03-30,8689,6893,2.48
1192,2020-08-24,101,298,3.89
72,2019-02-04,6525,683,2.27
952,2020-03-23,7118,0,NaN


In [5]:
#Vemos cuales son los helados existentes
numero_helados = len(df_ventas['id_producto'].unique())
total_ventas = df_ventas['ventas_unidades'].sum() 
print(f'El dataframe cuenta con {numero_helados} tipos de helados')
print(f"En total se han vendido {total_ventas} unidades")

El dataframe cuenta con 16 tipos de helados
En total se han vendido 3848068 unidades


¿Cuál es el identificador del helado con más ventas en euros?

In [6]:
df_ventas['ventas_euros'] = df_ventas['ventas_unidades']*df_ventas['precio']
print(f'El id del helado con mas ventas en euros es {df_ventas["ventas_euros"].idxmax()}')


El id del helado con mas ventas en euros es 104


In [7]:
df_ventas.loc[df_ventas['ventas_euros'].idxmax()]

fecha              2019-02-18 00:00:00
id_producto                        240
ventas_unidades                  95300
precio                            2.32
ventas_euros                  221096.0
Name: 104, dtype: object

En este caso no sé porque no da el mismo resultado

### Acceso a los datos de publicidad

Lee los datos de publicidad, y transformalos en una serie semanal con la suma de todos los medios.

In [8]:
df_inversiones

,fecha,inversion_tv,inversion_radio,inversion_youtube,inversion_facebook
0,2019-01-07,6100,0,6100,0
1,2019-01-14,0,0,0,6530
2,2019-01-21,1120,0,1120,5610
3,2019-01-28,0,0,0,1510
4,2019-02-04,880,880,880,0
...,...,...,...,...,...
99,2020-11-30,4880,4880,4880,0
100,2020-12-07,17580,17580,0,0
101,2020-12-14,26520,26520,26520,0
102,2020-12-21,860,0,860,4250


In [9]:
df_inversiones['fecha'] = pd.to_datetime(df_inversiones['fecha'])
df_semana = df_inversiones.groupby(df_inversiones['fecha'].dt.to_period('W'))
df_semana = df_semana.agg({
    'inversion_tv' : 'sum',
    'inversion_radio' : 'sum',
    'inversion_youtube' : 'sum',
    'inversion_facebook' : 'sum'
}).reset_index()

df_semana['suma_total'] = df_semana['inversion_tv']+df_semana['inversion_radio']+df_semana['inversion_youtube']+df_semana['inversion_facebook']

df_semana[['fecha','suma_total']]

,fecha,suma_total
0,2019-01-07/2019-01-13,12200
1,2019-01-14/2019-01-20,6530
2,2019-01-21/2019-01-27,7850
3,2019-01-28/2019-02-03,1510
4,2019-02-04/2019-02-10,2640
...,...,...
99,2020-11-30/2020-12-06,14640
100,2020-12-07/2020-12-13,35160
101,2020-12-14/2020-12-20,79560
102,2020-12-21/2020-12-27,5970


¿Cuál es el mes con más inversión total?

In [10]:
df_inversiones['fecha'] = pd.to_datetime(df_inversiones['fecha'])
df_mes = df_inversiones.groupby(df_inversiones['fecha'].dt.to_period('M'))
df_mes = df_mes.agg({
    'inversion_tv' : 'sum',
    'inversion_radio' : 'sum',
    'inversion_youtube' : 'sum',
    'inversion_facebook' : 'sum'
})

df_mes['suma_total'] = df_mes['inversion_tv']+df_mes['inversion_radio']+df_mes['inversion_youtube']+df_mes['inversion_facebook']

df_mes['suma_total'].idxmax()

Period('2020-08', 'M')

### Combinando datos

Combina el dataframe de ventas con las descripciones de los productos, y quédate sólo con los productos de helados de tarrina.

In [11]:
df_nombres.sample(5)


,id_producto,nombre
12,101,MAGNUM Helado White Chocolate & Cookies Tarrin...
3,5019,DIA Helado Dulce de Leche Tarrina 500 g
5,4468,NESTLE helado maxibon nata caja 4 uds 384 gr
0,4577,OREO helado sandwich caja 6 uds 198 gr
7,8689,NESTLE Helado cono kit kat classic caja 4 uds ...


In [12]:
df_ventas.sample(5)

,fecha,id_producto,ventas_unidades,precio,ventas_euros
313,2019-05-20,3636,345,2.40,828.00
452,2019-07-22,4577,4878,5.13,25024.14
740,2019-11-25,402,2394,2.99,7158.06
449,2019-07-22,4758,11832,4.82,57030.24
769,2019-12-09,4758,8647,4.79,41419.13


In [13]:
df_comb = df_ventas.merge(df_nombres)
df_comb

,fecha,id_producto,ventas_unidades,precio,ventas_euros,nombre
0,2019-01-07,4758,7953,4.82,38333.46,NESTLE helado maxibon cookies caja 4 uds 360 gr
1,2019-01-07,17,7052,1.50,10578.00,OREO Helado cono caja 4 uds 250 gr
2,2019-01-07,9134,6403,2.36,15111.08,DIA Helado cono sabor vainilla caja 6 uds 408 gr
3,2019-01-07,8689,4266,2.47,10537.02,NESTLE Helado cono kit kat classic caja 4 uds ...
4,2019-01-07,402,2932,2.89,8473.48,MILKA Helado cono chocolate y vainilla Caja 4 ...
...,...,...,...,...,...,...
1388,2020-12-28,5684,854,3.96,3381.84,DIA Helado Fresa Tarrina 1kg
1389,2020-12-28,101,758,3.31,2508.98,MAGNUM Helado White Chocolate & Cookies Tarrin...
1390,2020-12-28,3636,401,2.68,1074.68,CARTE D' OR Helado Stracciatella Tarrina 500 gr
1391,2020-12-28,7118,344,2.15,739.60,Haagen Dazs Helado Fresa Tarrina 400 g


In [14]:
df_tarrina = df_comb[df_comb['nombre'].str.contains('Tarrina', case=False, na=False)]
df_tarrina


,fecha,id_producto,ventas_unidades,precio,ventas_euros,nombre
5,2019-01-07,2033,1862,2.44,4543.28,CARTE D' OR Helado de yogur Tarrina 500 gr
8,2019-01-07,240,1015,2.30,2334.50,Helado Haagen Dazs Cookies Tarrina 400 g
10,2019-01-07,5684,317,3.70,1172.90,DIA Helado Fresa Tarrina 1kg
11,2019-01-07,101,208,3.93,817.44,MAGNUM Helado White Chocolate & Cookies Tarrin...
13,2019-01-07,3636,160,2.60,416.00,CARTE D' OR Helado Stracciatella Tarrina 500 gr
...,...,...,...,...,...,...
1388,2020-12-28,5684,854,3.96,3381.84,DIA Helado Fresa Tarrina 1kg
1389,2020-12-28,101,758,3.31,2508.98,MAGNUM Helado White Chocolate & Cookies Tarrin...
1390,2020-12-28,3636,401,2.68,1074.68,CARTE D' OR Helado Stracciatella Tarrina 500 gr
1391,2020-12-28,7118,344,2.15,739.60,Haagen Dazs Helado Fresa Tarrina 400 g


Agrega el dataset anterior en una serie semanal que agregue todos los productos mediante la suma de unidades y la media de los precios

In [15]:
df_comb

,fecha,id_producto,ventas_unidades,precio,ventas_euros,nombre
0,2019-01-07,4758,7953,4.82,38333.46,NESTLE helado maxibon cookies caja 4 uds 360 gr
1,2019-01-07,17,7052,1.50,10578.00,OREO Helado cono caja 4 uds 250 gr
2,2019-01-07,9134,6403,2.36,15111.08,DIA Helado cono sabor vainilla caja 6 uds 408 gr
3,2019-01-07,8689,4266,2.47,10537.02,NESTLE Helado cono kit kat classic caja 4 uds ...
4,2019-01-07,402,2932,2.89,8473.48,MILKA Helado cono chocolate y vainilla Caja 4 ...
...,...,...,...,...,...,...
1388,2020-12-28,5684,854,3.96,3381.84,DIA Helado Fresa Tarrina 1kg
1389,2020-12-28,101,758,3.31,2508.98,MAGNUM Helado White Chocolate & Cookies Tarrin...
1390,2020-12-28,3636,401,2.68,1074.68,CARTE D' OR Helado Stracciatella Tarrina 500 gr
1391,2020-12-28,7118,344,2.15,739.60,Haagen Dazs Helado Fresa Tarrina 400 g


In [16]:
df_semana_2 = df_comb.groupby(df_comb['fecha'].dt.to_period('W')).agg({
    'ventas_unidades' : 'sum',
    'precio' : 'mean'
})
df_semana_2.reset_index()

,fecha,ventas_unidades,precio
0,2019-01-07/2019-01-13,35155,2.890000
1,2019-01-14/2019-01-20,33580,2.866000
2,2019-01-21/2019-01-27,31990,2.848000
3,2019-01-28/2019-02-03,36750,2.862667
4,2019-02-04/2019-02-10,35868,2.930000
...,...,...,...
98,2020-11-30/2020-12-06,25153,2.669091
99,2020-12-07/2020-12-13,26069,2.623636
100,2020-12-14/2020-12-20,26503,2.640000
101,2020-12-21/2020-12-27,25394,2.612727


##### Combina todos los dataframes en uno solo (llamado dataset_conjunto), partiendo del dataset con los datos de helados de tarrina agregados por semana. Utiliza left joins

Crea una variable binaria <i>navidad</i> que valga 1 en las semanas de nochebuena y nochevieja


In [17]:
dataset_conjunto = df_tarrina.merge(df_comb)
dataset_conjunto['fecha'] = pd.to_datetime(dataset_conjunto['fecha'])
dataset_conjunto = dataset_conjunto.assign(navidad  = lambda x: (x['fecha'].dt.isocalendar().week == 52)|(x['fecha'].dt.isocalendar().week == 51))
dataset_conjunto['navidad'] = dataset_conjunto['navidad'].map({True:1 , False:0})
dataset_conjunto[dataset_conjunto['navidad']==1]

,fecha,id_producto,ventas_unidades,precio,ventas_euros,nombre,navidad
343,2019-12-16,2033,2045,2.49,5092.05,CARTE D' OR Helado de yogur Tarrina 500 gr,1
344,2019-12-16,240,1390,2.35,3266.50,Helado Haagen Dazs Cookies Tarrina 400 g,1
345,2019-12-16,3636,440,2.72,1196.80,CARTE D' OR Helado Stracciatella Tarrina 500 gr,1
346,2019-12-16,5684,425,3.46,1470.50,DIA Helado Fresa Tarrina 1kg,1
347,2019-12-16,101,308,3.67,1130.36,MAGNUM Helado White Chocolate & Cookies Tarrin...,1
348,2019-12-16,5019,153,2.47,377.91,DIA Helado Dulce de Leche Tarrina 500 g,1
349,2019-12-16,7118,0,NaN,NaN,Haagen Dazs Helado Fresa Tarrina 400 g,1
350,2019-12-23,2033,3574,2.48,8863.52,CARTE D' OR Helado de yogur Tarrina 500 gr,1
351,2019-12-23,240,1645,2.30,3783.50,Helado Haagen Dazs Cookies Tarrina 400 g,1
352,2019-12-23,5684,716,3.70,2649.20,DIA Helado Fresa Tarrina 1kg,1


### Jugando con expresiones regulares

¿Eres capaz de crear una variable de formato (peso en kilos) a partir del nombre del producto, utilizando expresiones regulares?

In [18]:
import re
df_kg = dataset_conjunto.copy()
peso = r'(\d+(\.\d+)?)\s*(g|kg)'
df_kg['peso'] = df_kg['nombre'].str.extract(peso, expand=False)[0]
df_kg['peso'] = df_kg['peso'] + df_kg['nombre'].str.extract(peso, expand=False)[2]
df_kg

,fecha,id_producto,ventas_unidades,precio,ventas_euros,nombre,navidad,peso
0,2019-01-07,2033,1862,2.44,4543.28,CARTE D' OR Helado de yogur Tarrina 500 gr,0,500g
1,2019-01-07,240,1015,2.30,2334.50,Helado Haagen Dazs Cookies Tarrina 400 g,0,400g
2,2019-01-07,5684,317,3.70,1172.90,DIA Helado Fresa Tarrina 1kg,0,1kg
3,2019-01-07,101,208,3.93,817.44,MAGNUM Helado White Chocolate & Cookies Tarrin...,0,300g
4,2019-01-07,3636,160,2.60,416.00,CARTE D' OR Helado Stracciatella Tarrina 500 gr,0,500g
...,...,...,...,...,...,...,...,...
716,2020-12-28,5684,854,3.96,3381.84,DIA Helado Fresa Tarrina 1kg,0,1kg
717,2020-12-28,101,758,3.31,2508.98,MAGNUM Helado White Chocolate & Cookies Tarrin...,0,300g
718,2020-12-28,3636,401,2.68,1074.68,CARTE D' OR Helado Stracciatella Tarrina 500 gr,0,500g
719,2020-12-28,7118,344,2.15,739.60,Haagen Dazs Helado Fresa Tarrina 400 g,0,400g


In [19]:
df_kg

,fecha,id_producto,ventas_unidades,precio,ventas_euros,nombre,navidad,peso
0,2019-01-07,2033,1862,2.44,4543.28,CARTE D' OR Helado de yogur Tarrina 500 gr,0,500g
1,2019-01-07,240,1015,2.30,2334.50,Helado Haagen Dazs Cookies Tarrina 400 g,0,400g
2,2019-01-07,5684,317,3.70,1172.90,DIA Helado Fresa Tarrina 1kg,0,1kg
3,2019-01-07,101,208,3.93,817.44,MAGNUM Helado White Chocolate & Cookies Tarrin...,0,300g
4,2019-01-07,3636,160,2.60,416.00,CARTE D' OR Helado Stracciatella Tarrina 500 gr,0,500g
...,...,...,...,...,...,...,...,...
716,2020-12-28,5684,854,3.96,3381.84,DIA Helado Fresa Tarrina 1kg,0,1kg
717,2020-12-28,101,758,3.31,2508.98,MAGNUM Helado White Chocolate & Cookies Tarrin...,0,300g
718,2020-12-28,3636,401,2.68,1074.68,CARTE D' OR Helado Stracciatella Tarrina 500 gr,0,500g
719,2020-12-28,7118,344,2.15,739.60,Haagen Dazs Helado Fresa Tarrina 400 g,0,400g


In [23]:
ruta = '/home/ayala/Escritorio/BOOTCAMP IA/SPRINT 0 - PROGRMACION EN PYTHON/LAB 4 - EXPLORATORIO DE DATOS EN PYTHON/EJERCICIOS/datos_helados_final.csv' 
dataset_conjunto.to_csv(ruta , index=False)